In [238]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

#### `.bib` entries loading

In [239]:
import bibtexparser

with open('../data/Knowledge.bib') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.


In [240]:
print(bib_database.entries[3]['author'])
print(bib_database.entries[3]['comment'])

Sun, Yue and Chen, Lihua and Yick, Kit-lun and Yu, Winnie and Lau, Newman and Jiao, Wanzhong
The FE model was supervised only by one variable: the increment of nipple points distance, which seems to be a very weak constraint. Would the whole workflow work when introducing all points tracked by the motion capture system, or even the ultra-dense motion capture system?


#### Group tree parsing

In [241]:
import re

def parse_group_tree(tree_str: str) -> dict:
    # extract group level and name and combined as an order list
    lines = re.split('\n', tree_str)
    group_list = []

    for line in lines:
        if 'Group' in line:
            level = int(line[0])
            if level == 0:
                group_list.append({
                    'level': level,
                    'name': 'Root',
                    'node': {'entries': []},
                    })
                
            else:
                name = re.search(r"(?<=Group:)[^\\;]+", line).group()  # at current stage only english group names are supported
                group_list.append({
                    'level': level, 
                    'name': name,
                    'node': {'entries': []},
                    })

    # prepare a dictionary of all groups for fast accessing via group name
    group_dict = {}

    for group in group_list:
        group_dict[group['name']] = group['node']

    # parse the group tree
    for idx_current in range(len(group_list)):
        # search backward for the 1st higher level node as current node's source node
        for idx_previous in range(idx_current, -1, -1):
            if group_list[idx_previous]['level'] < group_list[idx_current]['level']:
                source_node = group_list[idx_previous]['node']
                current_node = group_list[idx_current]['node']
                current_name = group_list[idx_current]['name']
                source_node[current_name] = current_node
                break
    
    tree_root = group_list[0]['node']

    return group_dict, tree_root

tree_str = bib_database.comments[1]
group_dict, tree_root = parse_group_tree(tree_str)

#### Attach each group with the corresponding entries

In [242]:
def attach_entries_to_group(entries: list, group_dict: dict):
    for entry in entries:
        # if the entry has groups attr, attach it to the corresponding groups
        if 'groups' in entry.keys():
            groups = re.split(', ', entry['groups'])

            for group in groups:
                group_dict[group]['entries'].append(entry)

        # if the entry doesn't have groups attr, attach it to the root group
        else:
            group_dict['Root']['entries'].append(entry)

attach_entries_to_group(bib_database.entries, group_dict)

#### Generate group pages

In [297]:
import os
import shutil

def export_group_pages(node: dict, name: str, source_name: str = "None", export_folder: str = 'output/'):
    # create group folder
    if os.path.exists(export_folder):
        shutil.rmtree(export_folder)
    os.mkdir(export_folder)

    # create group's markdown page
    md_path = os.path.join(export_folder, '{}.md'.format(name))
            
    with open(md_path, 'w') as file:
        # title section
        file.write("# {}\n".format(name))

        # roadmap section
        file.write("---\n")
        file.write("#### Roadmap\n\n")

        if name is not None:
            file.write("Source group: [[{}]]\n".format(source_name))

        file.write("Subgroups: \n\n")

        for subgroup in node.keys():
            if subgroup != "entries":
                file.write("- [[{}]]\n".format(subgroup))

        # entries section
        file.write("---\n")
        file.write("#### Entries\n\n")
        file.write("| Type | Note | Title |\n")
        file.write("| --- | --- | --- |\n")
        
        for entry in node['entries']:
            try:
                file.write("| {} | [[{}]] | {} |\n".format(
                entry['ENTRYTYPE'],
                entry['ID'],
                entry['title'],
                ))
            except:
                print("entry information miss:")
                print(entry)

    # recurse to the subgroups
    for subgroup in node.keys():
        if subgroup != "entries":
            sub_folder = os.path.join(export_folder, subgroup)
            export_group_pages(node[subgroup], subgroup, name, sub_folder)

export_group_pages(node=tree_root, name='Root', export_folder='../output/')